# convert `genies` datasets to [open_pref_eval](https://github.com/wassname/open_pref_eval)


Here I'm taking the GENIE datasets, and 
1. converting them to preference (compatible with open_pref_eval)
2. hosting on huggingface


## Setup

```sh
python -m venv .venv --prompt GENIES
. .venv/bin/activate
pip install wheel fire requests
pip install -r requirements.txt
python ./download_data.py
```

In [6]:
%reload_ext autoreload
%autoreload 2

In [7]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import datasets

from pathlib import Path
import json

In [8]:
path_to_distribution_shift_pairs = Path('../distribution_shifts/all.json')
pairs_data = json.load(open(path_to_distribution_shift_pairs))
pairs_data

[{'source': 'alpaca_easy', 'target': 'alpaca_hard'},
 {'source': 'arc_easy', 'target': 'arc_hard'},
 {'source': 'math_easy', 'target': 'math_hard'},
 {'source': 'code_easy', 'target': 'code_hard'},
 {'source': 'ranking_logic_easy', 'target': 'ranking_logic_hard'},
 {'source': 'raven_easy', 'target': 'raven_matrices'},
 {'source': 'alpaca_mmlu', 'target': 'spanish_input'},
 {'source': 'alpaca_mmlu', 'target': 'spanish_output'},
 {'source': 'alpaca_mmlu', 'target': 'comma_separated_input'},
 {'source': 'alpaca_mmlu', 'target': 'comma_separated_output'},
 {'source': 'alpaca_mmlu', 'target': 'ranking_logic'},
 {'source': 'alpaca_mmlu', 'target': 'raven_matrices'},
 {'source': 'alpaca_mmlu', 'target': 'word_swap'},
 {'source': 'code', 'target': 'counterfactual_python'},
 {'source': 'code', 'target': 'us_history'},
 {'source': 'code', 'target': 'change_my_view'},
 {'source': 'cooking', 'target': 'math'},
 {'source': 'cooking', 'target': 'raven_matrices'},
 {'source': 'math', 'target': 'chang

In [9]:
from datasets import DatasetInfo, Dataset

def genie2ds(train: list) -> pd.DataFrame:
    """takes the GENIE format and convert it to to a dataframe of preference format."""
    outs = []
    for i, row in enumerate(train):
        s = pd.Series(row['responses'])
        chosen = s[s==1].index[0]
        rejected = s[s==0].index
        outs += [dict(prompt=row['prompt'], chosen=chosen, rejected=r, i=i) for r in rejected]

    df = pd.DataFrame(outs)
    return df



def json2ds(source_dir: Path) -> Dataset:
    test = json.load(open(source_dir / 'test.json'))
    train = json.load(open(source_dir / 'train.json'))
    metadata = json.load(open(source_dir / 'metadata.json'))
    ds_info = DatasetInfo(
        description= f"GENIE:{metadata['id']}",
        citation= """@misc{clymer2023generalizationanalogiestestbedgeneralizing,
        title={Generalization Analogies: A Testbed for Generalizing AI Oversight to Hard-To-Measure Domains}, 
        author={Joshua Clymer and Garrett Baker and Rohan Subramani and Sam Wang},
        year={2023},
        eprint={2311.07723},
        archivePrefix={arXiv},
        primaryClass={cs.AI},
        url={https://arxiv.org/abs/2311.07723}, 
    }""",
        homepage= "https://joshuaclymer.github.io/generalization-analogies-website/",
        license= "MIT",
        config_name=f"{metadata['id']}",
    )


    df_train = genie2ds(train)
    df_test = genie2ds(test)
    dataset2 = datasets.DatasetDict(
            {'train': datasets.Dataset.from_pandas(df_train, info=ds_info),
                'test': datasets.Dataset.from_pandas(df_test, info=ds_info)}
        )
    return dataset2

In [10]:
dist_dir = Path('../distributions')

for pair in  pairs_data:
    for key in ['source', 'target']:
        source_dir = dist_dir / pair[key]
        metadata = json.load(open(source_dir / 'metadata.json'))
        config_name = metadata['id']

        # first check if it exists
        try:
            load_dataset(f"wassname/genie_dpo", name=config_name, keep_in_memory=False, split='train[:1]')
        except ValueError as e:
            print(e)
            print(f"Dataset {config_name} does not exist, creating")

            dataset2 = json2ds(source_dir)
            print(source_dir, config_name, dataset2)

            dataset2.push_to_hub("wassname/genie_dpo", config_name=config_name)
        else:
            print(f"Dataset {config_name} already exists, skipping")

        

Dataset alpaca_easy already exists, skipping
Dataset alpaca_hard already exists, skipping
Dataset arc_easy already exists, skipping
Dataset arc_hard already exists, skipping
Dataset math_easy already exists, skipping
Dataset math_hard already exists, skipping
Dataset code_easy already exists, skipping
Dataset code_hard already exists, skipping
Dataset ranking_logic_easy already exists, skipping
Dataset ranking_logic_hard already exists, skipping
Dataset raven_easy already exists, skipping
Dataset raven_matrices already exists, skipping
Dataset alpaca_mmlu already exists, skipping
Dataset spanish_input already exists, skipping
Dataset alpaca_mmlu already exists, skipping
Dataset spanish_output already exists, skipping
Dataset alpaca_mmlu already exists, skipping
Dataset comma_separated_input already exists, skipping
Dataset alpaca_mmlu already exists, skipping
Dataset comma_separated_output already exists, skipping
Dataset alpaca_mmlu already exists, skipping
Dataset ranking_logic alrea

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

BuilderConfig 'alpaca_long' not found. Available: ['alpaca_chat', 'alpaca_easy', 'alpaca_hard', 'alpaca_high_quality', 'alpaca_low_quality', 'alpaca_mmlu', 'alpaca_short', 'arc_easy', 'arc_hard', 'biology_with_literary_style', 'change_my_view', 'code', 'code_easy', 'code_hard', 'code_low_quality', 'comma_separated_input', 'comma_separated_output', 'cooking', 'counterfactual_python', 'creative_writing', 'crt_1', 'crt_2', 'crt_3', 'gender_bias', 'math', 'math_easy', 'math_fiction', 'math_hard', 'math_make_questions', 'math_textbook', 'personality_traits', 'punishment_avoidance', 'pursue_goals', 'ranking_logic', 'ranking_logic_easy', 'ranking_logic_hard', 'raven_easy', 'raven_matrices', 'relinquish_power', 'reward_seeking', 'shp_high_quality', 'shp_low_quality', 'spanish_input', 'spanish_output', 'survival_influence', 'sycophancy_answer', 'sycophancy_are_you_sure', 'sycophancy_feedback', 'sycophancy_mimicry', 'truthful_qa', 'us_history', 'us_history_fiction', 'us_history_make_questions', 

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]